In [1]:
import copy
import numpy as np
import matplotlib.pyplot as plt
import tqdm

In [2]:
#helper functions from: https://github.com/meichlseder/pyascon/tree/1ad51b091d2233a29beb2e7b5136f791cf5100ff

def ascon_initialize(S, k, rate, a, b, key, nonce):
    """
    Ascon initialization phase - internal helper function.
    S: Ascon state, a list of 5 64-bit integers
    k: key size in bits
    rate: block size in bytes (8 for Ascon-128, Ascon-80pq; 16 for Ascon-128a)
    a: number of initialization/finalization rounds for permutation
    b: number of intermediate rounds for permutation
    key: a bytes object of size 16 (for Ascon-128, Ascon-128a; 128-bit security) or 20 (for Ascon-80pq; 128-bit security)
    nonce: a bytes object of size 16
    returns nothing, updates S
    """
    iv_zero_key_nonce = to_bytes([k, rate * 8, a, b] + (20-len(key))*[0]) + key + nonce
    S[0], S[1], S[2], S[3], S[4] = bytes_to_state(iv_zero_key_nonce)

    S_input = copy.deepcopy(S)
    ascon_permutation(S, 12) #just first round

    S_output = S
    return S_input, S_output

def ascon_permutation(S, rounds=1):
    """
    Ascon core permutation for the sponge construction - internal helper function.
    S: Ascon state, a list of 5 64-bit integers
    rounds: number of rounds to perform
    returns nothing, updates S
    """
    assert(rounds <= 12)
    if debugpermutation: printwords(S, "permutation input:")
        
    for r in range(12-rounds, 12):
        # --- add round constants ---
        S[2] ^= (0xf0 - r*0x10 + r*0x1)
        if debugpermutation: printwords(S, "round constant addition:")
        # --- substitution layer ---
        S[0] ^= S[4]
        S[4] ^= S[3]
        S[2] ^= S[1]
        T = [(S[i] ^ 0xFFFFFFFFFFFFFFFF) & S[(i+1)%5] for i in range(5)]
        for i in range(5):
            S[i] ^= T[(i+1)%5]
        S[1] ^= S[0]
        S[0] ^= S[4]
        S[3] ^= S[2]
        S[2] ^= 0XFFFFFFFFFFFFFFFF
        if debugpermutation: printwords(S, "substitution layer:")
        # --- linear diffusion layer ---
        S[0] ^= rotr(S[0], 19) ^ rotr(S[0], 28)
        S[1] ^= rotr(S[1], 61) ^ rotr(S[1], 39)
        S[2] ^= rotr(S[2],  1) ^ rotr(S[2],  6)
        S[3] ^= rotr(S[3], 10) ^ rotr(S[3], 17)
        S[4] ^= rotr(S[4],  7) ^ rotr(S[4], 41)
        if debugpermutation: printwords(S, "linear diffusion layer:")
                        
# === helper functions ===

def get_random_bytes(num):
    import os
    return to_bytes(os.urandom(num))

def zero_bytes(n):
    return n * b"\x00"

def to_bytes(l): # where l is a list or bytearray or bytes
    return bytes(bytearray(l))

def bytes_to_int(bytes):
    return sum([bi << ((len(bytes) - 1 - i)*8) for i, bi in enumerate(to_bytes(bytes))])

def bytes_to_state(bytes):
    return [bytes_to_int(bytes[8*w:8*(w+1)]) for w in range(5)]

def int_to_bytes(integer, nbytes):
    return to_bytes([(integer >> ((nbytes - 1 - i) * 8)) % 256 for i in range(nbytes)])

def rotr(val, r):
    return (val >> r) | ((val & (1<<r)-1) << (64-r))

def bytes_to_hex(b):
    return b.hex()
    #return "".join(x.encode('hex') for x in b)

def printstate(S, description=""):
    print(" " + description)
    print(" ".join(["{s:016x}".format(s=s) for s in S]))

def printwords(S, description=""):
    print(" " + description)
    print("\n".join(["  x{i}={s:016x}".format(**locals()) for i, s in enumerate(S)]))

# === some demo if called directly ===

def demo_print(data):
    maxlen = max([len(text) for (text, val) in data])
    for text, val in data:
        print("{text}:{align} 0x{val} ({length} bytes)".format(text=text, align=((maxlen - len(text)) * " "), val=bytes_to_hex(val), length=len(val)))

In [3]:
# added helper functions
def state_to_int320(state):
    state_int320 = (state[0]<<4*64)+(state[1]<<3*64)+(state[2]<<2*64)+(state[3]<<64)+state[4]
    return state_int320

In [4]:
variant          = "Ascon-128"
debug            = 0
debugpermutation = 1
keysize          = 20 if variant == "Ascon-80pq" else 16

a                = 12 # rounds
b                = 8  if variant == "Ascon-128a" else 6   # rounds
rate             = 16 if variant == "Ascon-128a" else 8   # rate in bytes

In [5]:
# choose a cryptographically strong random key 
key = get_random_bytes(keysize) # zero_bytes(keysize)
# key = b'\xc3\xd5\x90\x80\xd3N\x9d\x17\xcb\x843\xe8\xcb\x0f\xfb\xe6'

In [6]:
key   = int_to_bytes(int("2b7e151628aed2a6abf7158809cf4f3c",16),16)
nonce = int_to_bytes(int("ee4358ac98edba7348e078c3e577aa55",16),16)

In [7]:
S                 = [0, 0, 0, 0, 0]
k                 = len(key) * 8
S_input, S_output = ascon_initialize(S, k, rate, a, b, key, nonce)

 permutation input:
  x0=80400c0600000000
  x1=2b7e151628aed2a6
  x2=abf7158809cf4f3c
  x3=ee4358ac98edba73
  x4=48e078c3e577aa55
 round constant addition:
  x0=80400c0600000000
  x1=2b7e151628aed2a6
  x2=abf7158809cf4fcc
  x3=ee4358ac98edba73
  x4=48e078c3e577aa55
 substitution layer:
  x0=cddc55309124e799
  x1=0d1d397b541457aa
  x2=7fd6df22bb8c6291
  x3=266a2cf15cfb8d4c
  x4=85fd217d75124084
 linear diffusion layer:
  x0=43611517c647a6b4
  x1=935cda0ea0acd088
  x2=85c2ebcf6ca46253
  x3=b3c5a54f76d41dd2
  x4=324c521fcdba9a95
 round constant addition:
  x0=43611517c647a6b4
  x1=935cda0ea0acd088
  x2=85c2ebcf6ca462b2
  x3=b3c5a54f76d41dd2
  x4=324c521fcdba9a95
 substitution layer:
  x0=7676099f5c9359dc
  x1=47b238c9d585c35b
  x2=e9699c2ebaddcfc0
  x3=d577c696336715cd
  x4=03d96f561b6e47cf
 linear diffusion layer:
  x0=d4781a96fac02bbb
  x1=e988f501cf24bdf0
  x2=9e78f4495d585f1f
  x3=2ce471dc75a0d5bb
  x4=36d36aab50d977f7
 round constant addition:
  x0=d4781a96fac02bbb
  x1=e988f501cf24b

In [8]:
res = [61, 137, 6, 100, 19, 174, 74, 154, 71, 129, 196, 219, 83, 35, 48, 195]

for i in range(len(res)):
    print(hex(res[i]))

0x3d
0x89
0x6
0x64
0x13
0xae
0x4a
0x9a
0x47
0x81
0xc4
0xdb
0x53
0x23
0x30
0xc3
